<a href="https://colab.research.google.com/github/marcory-hub/hailo/blob/main/colab_yolov8s_create_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. How to prepair you dataset

1. In your Google Drive make a folder 'datasets'. Download the images and labels to that folder. If you do not have your own dataset, you can download the [hornet3000+ dataset](https://www.kaggle.com/datasets/marcoryvandijk/vespa-velutina-v-crabro-vespulina-vulgaris)

2. Folder structure
Only if you use your own dataset: make sure it has this folder structure with these exact names of the folders (class_1, class_2 can have your names of the classes):

```
data
  train
    images
      class_1
      class_2
      etc...
    labels
      class_1
      class_2
      ect...
   val
    images
      class_1
      class_2
      ...
    labels
      class_1
      class_2
      ...
```

3. Only if you use own dataset: zip the data folder to a file names `data.zip`.

4. Copy the data.zip file (with the Kaggle dataset) to the folder datasets.

5. Add dataset.yaml to folder datasets. This contains:
- absolute path to train images (train)
- absolute path validation (val)
- the number of classes (nc)
- and the class names (names)
For example:
```
train: /content/data/train # train images
val: /content/data/val # val images
nc: 3
names: ['Vespa_velutina', 'Vespa_crabro', 'Vespula_vulgaris']
```
If you use your own dataset make sure to adjust the nc and names accordingly.

# 2. Mount your Google Drive

In [ ]:
# Mount Google Drive

from google.colab import drive

drive.mount('/content/gdrive')

Check GPU (optional)

In [ ]:
# Check GPU (make sure runtimetype GPU (T4 or A100) is selected)
# If 'command not found' then change the runtimetype (click 'Runtime' and select change runtimetype in the drop down menu)

!nvidia-smi

# 3. Copy data.zip to Colab file system and unzip

In [ ]:
# Prepare data

!scp '/content/gdrive/My Drive/datasets/data.zip' '/content/data.zip'

!unzip '/content/data.zip'

Check paths (optional)

In [ ]:
# Check is path is correct

#  output should be
#     train val
#     images labels
#     dataset.yaml data.zip

# If not, change your datastructure as described under 1.

!ls '/content/data/'

!ls '/content/data/train/'

!ls '/content/data/val/'

!ls '/content/gdrive/My Drive/datasets/'

# 4. Install libraries and train the model

In [ ]:
# Pip install method (recommended)

!pip install ultralytics==8.0.196

# Fix for "TypeError: unhashable type: 'numpy.ndarray'"

!pip install albumentations==1.4

In [ ]:
# Train the model

# Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
# wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
# wandb: You can find your API key in your browser here: https://wandb.ai/authorize

import os

from ultralytics import YOLO

# Load a model
model = YOLO("yolov8s.pt")  # load a pretained model (choose yolov8n for the smallest model)

# Use the model
# Adjust epochs, batch, imgsz or other arguments if needed to optimize the model
results = model.train(data='/content/gdrive/MyDrive/datasets/dataset.yaml', epochs=1, batch=16, imgsz=640)  # train the model

# 5. Export the model in ONNX format

In [ ]:
# Export the model in ONNX format

model.export(format="onnx", opset=9)


In [ ]:
# Export the model in ONNX format IR_version 9 (adjust IR version if needed)

import onnx

# Load the ONNX model
model = onnx.load("/content/runs/detect/train/weights/best.onnx")

# Change the IR version to 9
model.ir_version = 9

# Save the modified model
onnx.save(model, "/content/runs/detect/train/weights/best9.onnx")

print("Model IR version changed to 9 and saved as best9.onnx")


# 6. Zip and Download the results to your local computer

Data is lost when closing the page!!!

## If an error occurs:
1. Click `Runtime` in the toolbar on the top
2. Select `Restart session` / `Sessie opnieuw starten` from the drop down menu
3. Run this codeblock again

In [ ]:
# DON'T FORGET TO DOWNLOAD THE RESULTS!!!

# Data is lost when closing the page!!!

# IN MOST CASES AN ERROR OCCURS:
# Click `Runtime`, select `Restart session` / `Sessie opnieuw opstarten`
# then run this codeblock again

from google.colab import files

try:
  !zip -r /content/runs.zip /content/runs
  files.download('/content/runs.zip')
except Exception as e:
  print(f"An error occurred: {e}")
  print("Click 'Runtime' -> 'Restart session' and try running the code again.")

# 7. [Compile the model in the Google Cloud Platform VM](https://github.com/marcory-hub/hailo/blob/main/compile-the-model-using-hailo-model-zoo.md).


